In [21]:
import os

from ladybug.sql import SQLiteResult

from eppy import *
from eppy.modeleditor import IDF
from eppy.results import readhtml
from eppy.results import fasthtml


import pickle

import plotly.graph_objects as go
import plotly.express as px

import pandas as pd

import pprint

import numpy as np


from ladybug import analysisperiod as ap

import datetime as dt
import json
from plotly.subplots import make_subplots
from collections import OrderedDict


In [3]:
root = os.getcwd()
shared_drive_root = "/Users/julietnwagwuume-ezeoke/My Drive/CEE256_BuildSys/final_256/Data/Calibration/group_models"
model_dir = os.path.join(shared_drive_root, "Final_project_3-6-2022_EH6")
model_dir


'/Users/julietnwagwuume-ezeoke/My Drive/CEE256_BuildSys/final_256/Data/Calibration/group_models/Final_project_3-6-2022_EH6'

# Adjust IDF

In [4]:
# # make a new dir for the output
adjusted_model_dir = os.path.join(root, "eppy_adjusted_models_2")
## NEED TO CHANGE THE LAST STRING TO CREATE NEW FOLDER!!!
new_dir_name = os.path.join(adjusted_model_dir, "final_pres_03_06_new_vars_3")


In [4]:
# set the idd
iddfile = "/Applications/OpenStudioApplication-1.1.1/EnergyPlus/Energy+.idd"
IDF.setiddname(iddfile)

# get the idf 
idfPath = os.path.join(model_dir, "in.idf")
epw = "/Users/julietnwagwuume-ezeoke/Documents/cee256_local/weather_files/CA_PALO-ALTO-AP_724937S_19.epw"
idf0 = IDF(idfPath, epw)

intVarsPath = os.path.join(root, "intVars")
with open(intVarsPath, "rb") as f:
    intVars = pickle.load(f)
intVars

# update the outputs with interesting variables 
currVars = [var.Variable_Name for var in idf0.idfobjects["Output:Variable"]]
for var in intVars:
    if var not in currVars:
        newObj = idf0.newidfobject("Output:Variable", Key_Value='', Variable_Name=var)

# change all reporting frequency
allVars = [var for var in idf0.idfobjects["Output:Variable"]]
for var in allVars:
    var.Reporting_Frequency = "Hourly"

allVars = [var for var in idf0.idfobjects["Output:Meter"]]
for var in allVars:
    var.Reporting_Frequency = "Monthly"

# change the run period
runPeriod = idf0.idfobjects["RunPeriod"][0]
runPeriod.Begin_Month = 1 
runPeriod.Begin_Day_of_Month = 1
runPeriod.End_Month = 12
runPeriod.End_Day_of_Month = 31

# remove extraneous electricity:facility terms 
# for some reason openstudio adds three of them which confuses
# the program that reads sql
extra_elect = [var for var in idf0.idfobjects["Output:Meter:MeterFileOnly"]]
extra_elect


for e in extra_elect:
  idf0.removeidfobject(e)
check = [var for var in idf0.idfobjects["Output:Meter:MeterFileOnly"]]
# should see a blank array showing these were removed
print(check)



[]


In [10]:
# os.makedirs(new_dir_name)

# # save the updated idf there
# idf0.save(os.path.join(new_dir_name, "in2.idf"))

# # run the idf 
# idf0.run(output_directory=new_dir_name)

# Simulation Analyis

## Load html 

In [12]:
html_path = os.path.join(new_dir_name, "eplustbl.htm")
filehandle = open(html_path, 'r') # get a file handle to the html file
namedtable = fasthtml.tablebyname(filehandle, "Tariff Summary")
pp = pprint.PrettyPrinter()
pp.pprint(namedtable)


['Tariff Summary',
 [['',
   'Selected',
   'Qualified',
   'Meter',
   'Buy or Sell',
   'Group',
   'Annual Cost ($)'],
  ['ELECTRICITY TARIFF',
   'Yes',
   'Yes',
   'ELECTRICITYPURCHASED:FACILITY',
   'Buy',
   '(none)',
   19107.05],
  ['GAS TARIFF', 'Yes', 'Yes', 'NATURALGAS:FACILITY', 'Buy', '(none)', 0.0],
  ['WATER TARIFF', 'No', 'Yes', 'WATER:FACILITY', 'Buy', '(none)', 0.0],
  ['DISTRICTHEATING TARIFF',
   'Yes',
   'Yes',
   'DISTRICTHEATING:FACILITY',
   'Buy',
   '(none)',
   2580.64],
  ['DISTRICTCOOLING TARIFF',
   'Yes',
   'Yes',
   'DISTRICTCOOLING:FACILITY',
   'Buy',
   '(none)',
   10333.85]]]


In [22]:
table_rows = namedtable[1]

costs = OrderedDict({
    "elect": [t for t in table_rows if t[0] == 'ELECTRICITY TARIFF'][0][-1],
    "hot": [t for t in table_rows if t[0] == 'DISTRICTHEATING TARIFF'][0][-1],
    "chill": [t for t in table_rows if t[0] == 'DISTRICTCOOLING TARIFF'][0][-1],
})
costs["total"] = costs["elect"] +  costs["hot"] +  costs["chill"] 
costs

OrderedDict([('elect', 19107.05),
             ('hot', 2580.64),
             ('chill', 10333.85),
             ('total', 32021.54)])

In [ ]:
namedtable = fasthtml.tablebyname(filehandle, "Tariff Summary")
pp = pprint.PrettyPrinter()
pp.pprint(namedtable)

## Load SQL File

In [5]:
# access the sql file 
sqlFile = os.path.join(new_dir_name, "eplusout.sql")
sqld = SQLiteResult(sqlFile)
sql_outputs_info_0 = sqld.available_outputs_info
sql_outputs_info = {item["output_name"]: {"name": item["output_name"], "units": item["units"], "data_type": item["data_type"]} for item in sql_outputs_info_0 }
sql_outputs_info


{'Zone People Total Heating Rate': {'name': 'Zone People Total Heating Rate',
  'units': 'W',
  'data_type': Power},
 'Zone Windows Total Heat Gain Energy': {'name': 'Zone Windows Total Heat Gain Energy',
  'units': 'kWh',
  'data_type': Energy},
 'Surface Window Heat Gain Energy': {'name': 'Surface Window Heat Gain Energy',
  'units': 'kWh',
  'data_type': Energy},
 'Heating Coil Heating Energy': {'name': 'Heating Coil Heating Energy',
  'units': 'kWh',
  'data_type': Energy},
 'Zone Air Terminal Outdoor Air Volume Flow Rate': {'name': 'Zone Air Terminal Outdoor Air Volume Flow Rate',
  'units': 'm3/s',
  'data_type': Volume Flow Rate},
 'DistrictHeating:Facility': {'name': 'DistrictHeating:Facility',
  'units': 'kWh',
  'data_type': Energy},
 'Electricity:Facility': {'name': 'Electricity:Facility',
  'units': 'kWh',
  'data_type': Energy},
 'Zone Operative Temperature': {'name': 'Zone Operative Temperature',
  'units': 'C',
  'data_type': Temperature},
 'District Heating Hot Water Ra

In [9]:
heating_collection = sqld.data_collections_by_output_name("DistrictHeating:Facility")[-1]
heating_collection.total

18907.775536645266

In [ ]:
hours = [dt.datetime(2019, 1, 1, i ).strftime("%H:%M") for i in range(24)]


In [ ]:
def month_by_hour_zonal_plot(data_name, month_num):
    zone_data = sqld.data_collections_by_output_name_run_period(data_name, 7)
    zonal_data  = {ix: value.average_monthly_per_hour() for ix, value in enumerate(zone_data)}

    month = ap.AnalysisPeriod(st_month=month_num, end_month=month_num)

    data_type = sql_outputs_info[data_name]["data_type"]
    units = sql_outputs_info[data_name]["units"]

    fig = go.Figure()

    for k, v in zonal_data.items():
            fig.add_trace(
                go.Scatter(x=hours, y=v.filter_by_analysis_period(month).values, mode='lines+markers', marker_size=25-5*(k+1), name=f"Zone {k+1}")
            )

    fig.update_layout(
        title=f"{data_name} Month #{month_num}",
        xaxis_title="Hours",
        yaxis_title=f"{data_type} [{units}]",
    )
    fig.show()

In [ ]:
sql_outputs_info


## Plots

In [ ]:
month_by_hour_zonal_plot("Zone Windows Total Transmitted Solar Radiation Energy", 8)
# zone 3 and 4 are the top 

In [ ]:
month_by_hour_zonal_plot("Zone Air Terminal Outdoor Air Volume Flow Rate", 8)

In [ ]:
month_by_hour_zonal_plot("Zone Infiltration Total Heat Gain Energy", 8)

In [ ]:

month_by_hour_zonal_plot("Zone Lights Total Heating Rate", 8)

In [ ]:

month_by_hour_zonal_plot("Surface Window Heat Gain Rate", 8)

In [ ]:
month_by_hour_zonal_plot("Zone Air Terminal Minimum Air Flow Fraction", 8)

In [ ]:
month_by_hour_zonal_plot("Zone Electric Equipment Total Heating Rate", 8)

In [ ]:
month_by_hour_zonal_plot("Zone People Total Heating Rate", 8)

In [ ]:
month_by_hour_zonal_plot("Heating Coil Heating Rate", 1)

In [ ]:
month_by_hour_zonal_plot("Heating Coil Heating Rate", 8)

In [ ]:
month_by_hour_zonal_plot("Zone Mean Air Temperature", 1)

In [ ]:
month_by_hour_zonal_plot("Zone Mean Air Temperature", 8)

In [ ]:
month_by_hour_zonal_plot("Zone Air Terminal VAV Damper Position", 1)

In [ ]:
month_by_hour_zonal_plot("Zone Air Terminal VAV Damper Position", 8)

In [ ]:
month_by_hour_zonal_plot("Zone Air Heat Balance Surface Convection Rate", 1)

In [ ]:
month_by_hour_zonal_plot("Zone Total Internal Total Heating Rate", 1)

# HVAC Sizing 

In [ ]:
hvacFile = os.path.join(new_dir_name, "eplusssz.csv")
hvac_sizing = pd.read_csv(hvacFile)
hvac_sizing

,Time,VAV WITH PFP BOXES AND REHEAT:DesPer 1:Des Heat Mass Flow [kg/s],VAV WITH PFP BOXES AND REHEAT:DesPer 1:Des Heat Cap [W],VAV WITH PFP BOXES AND REHEAT:DesPer 1:Des Cool Mass Flow [kg/s],VAV WITH PFP BOXES AND REHEAT:DesPer 1:Des Sens Cool Cap [W],VAV WITH PFP BOXES AND REHEAT:DesPer 1:Des Tot Cool Cap [W],VAV WITH PFP BOXES AND REHEAT:DesPer 2:Des Heat Mass Flow [kg/s],VAV WITH PFP BOXES AND REHEAT:DesPer 2:Des Heat Cap [W],VAV WITH PFP BOXES AND REHEAT:DesPer 2:Des Cool Mass Flow [kg/s],VAV WITH PFP BOXES AND REHEAT:DesPer 2:Des Sens Cool Cap [W],...,VAV WITH PFP BOXES AND REHEAT:DesPer 5:Des Heat Mass Flow [kg/s],VAV WITH PFP BOXES AND REHEAT:DesPer 5:Des Heat Cap [W],VAV WITH PFP BOXES AND REHEAT:DesPer 5:Des Cool Mass Flow [kg/s],VAV WITH PFP BOXES AND REHEAT:DesPer 5:Des Sens Cool Cap [W],VAV WITH PFP BOXES AND REHEAT:DesPer 5:Des Tot Cool Cap [W],VAV WITH PFP BOXES AND REHEAT:DesPer 6:Des Heat Mass Flow [kg/s],VAV WITH PFP BOXES AND REHEAT:DesPer 6:Des Heat Cap [W],VAV WITH PFP BOXES AND REHEAT:DesPer 6:Des Cool Mass Flow [kg/s],VAV WITH PFP BOXES AND REHEAT:DesPer 6:Des Sens Cool Cap [W],VAV WITH PFP BOXES AND REHEAT:DesPer 6:Des Tot Cool Cap [W]
0,00:15:00,1.355793,0.000000,2.153525,32110.34,30568.65,1.355793,0.0,1.577374,8778.733,...,2.184809,33589.99,1.355793,0.0,0.0,1.651020,7465.688,1.355793,0.0,0.0
1,00:30:00,1.355793,0.000000,2.155340,31913.24,30365.99,1.355793,0.0,1.581152,8635.312,...,2.184812,33590.03,1.355793,0.0,0.0,1.651023,7465.700,1.355793,0.0,0.0
2,00:45:00,1.355793,0.000000,2.156101,31700.26,30148.21,1.355793,0.0,1.583847,8485.307,...,2.184815,33590.08,1.355793,0.0,0.0,1.651025,7465.711,1.355793,0.0,0.0
3,01:00:00,1.355793,0.000000,2.156069,31475.55,29919.27,1.355793,0.0,1.585727,8330.458,...,2.184817,33590.12,1.355793,0.0,0.0,1.651027,7465.720,1.355793,0.0,0.0
4,01:15:00,1.355793,0.000000,2.157015,31339.82,29780.00,1.355793,0.0,1.587755,8231.028,...,2.184825,33590.24,1.355793,0.0,0.0,1.651037,7465.762,1.355793,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,23:30:00,1.355793,0.000000,2.132789,32540.55,31027.76,1.355793,0.0,1.551874,9174.813,...,2.184786,33589.64,1.355793,0.0,0.0,1.650993,7465.565,1.355793,0.0,0.0
94,23:45:00,1.355793,0.000000,2.143349,32441.58,30916.38,1.355793,0.0,1.563860,9055.919,...,2.184796,33589.79,1.355793,0.0,0.0,1.651005,7465.621,1.355793,0.0,0.0
95,24:00:00,1.355793,0.000000,2.150234,32284.90,30749.81,1.355793,0.0,1.572080,8912.767,...,2.184805,33589.93,1.355793,0.0,0.0,1.651017,7465.673,1.355793,0.0,0.0
96,Coinc Peak,2.184923,4.983134,33591.740000,71710.19,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
hvac_sizing.columns

Index(['Time',
       'VAV WITH PFP BOXES AND REHEAT:DesPer 1:Des Heat Mass Flow [kg/s]',
       'VAV WITH PFP BOXES AND REHEAT:DesPer 1:Des Heat Cap [W]',
       'VAV WITH PFP BOXES AND REHEAT:DesPer 1:Des Cool Mass Flow [kg/s]',
       'VAV WITH PFP BOXES AND REHEAT:DesPer 1:Des Sens Cool Cap [W]',
       'VAV WITH PFP BOXES AND REHEAT:DesPer 1:Des Tot Cool Cap [W]',
       'VAV WITH PFP BOXES AND REHEAT:DesPer 2:Des Heat Mass Flow [kg/s]',
       'VAV WITH PFP BOXES AND REHEAT:DesPer 2:Des Heat Cap [W]',
       'VAV WITH PFP BOXES AND REHEAT:DesPer 2:Des Cool Mass Flow [kg/s]',
       'VAV WITH PFP BOXES AND REHEAT:DesPer 2:Des Sens Cool Cap [W]',
       'VAV WITH PFP BOXES AND REHEAT:DesPer 2:Des Tot Cool Cap [W]',
       'VAV WITH PFP BOXES AND REHEAT:DesPer 3:Des Heat Mass Flow [kg/s]',
       'VAV WITH PFP BOXES AND REHEAT:DesPer 3:Des Heat Cap [W]',
       'VAV WITH PFP BOXES AND REHEAT:DesPer 3:Des Cool Mass Flow [kg/s]',
       'VAV WITH PFP BOXES AND REHEAT:DesPer 3:Des Sens

In [ ]:
fig = go.Figure()


for i in range(6):
    fig.add_trace(
            go.Scatter(x=hvac_sizing.Time, y=hvac_sizing[f"VAV WITH PFP BOXES AND REHEAT:DesPer {i+1}:Des Tot Cool Cap [W]"], mode='lines+markers', 
            name=f"Cooling Period {i+1}"
            ))
# fig.add_trace(
#             go.Scatter(x=hvac_sizing.Time, y=hvac_sizing["VAV WITH PFP BOXES AND REHEAT:DesPer 5:Des Heat Cap [W]"], mode='lines+markers', 
#             name="Heating Period 5 "
#             ))

# fig.add_trace(
#             go.Scatter(x=hvac_sizing.Time, y=hvac_sizing["VAV WITH PFP BOXES AND REHEAT:DesPer 6:Des Heat Cap [W]"], mode='lines+markers', 
#             name="Heating Period 6"
#             ))

fig.update_layout(
    title=f"cooling cap",
    xaxis_title="Hours",
    yaxis_title=f"[W]",
)
fig.show()

In [ ]:
fig = go.Figure()

for i in range(6):
    fig.add_trace(
            go.Scatter(x=hvac_sizing.Time, y=hvac_sizing[f"VAV WITH PFP BOXES AND REHEAT:DesPer {i+1}:Des Heat Mass Flow [kg/s]"], mode='lines+markers', 
            name=f"Heating Period {i+1}"
            ))
# fig.add_trace(
#             go.Scatter(x=hvac_sizing.Time, y=hvac_sizing["VAV WITH PFP BOXES AND REHEAT:DesPer 5:Des Heat Cap [W]"], mode='lines+markers', 
#             name="Heating Period 5 "
#             ))

# fig.add_trace(
#             go.Scatter(x=hvac_sizing.Time, y=hvac_sizing["VAV WITH PFP BOXES AND REHEAT:DesPer 6:Des Heat Cap [W]"], mode='lines+markers', 
#             name="Heating Period 6"
#             ))

fig.update_layout(
    title=f"Mass Flow",
    xaxis_title="Hours",
    yaxis_title=f"Mass Flow [kg/s]",
)
fig.show()

In [ ]:
fig = go.Figure()


fig.add_trace(
            go.Scatter(x=hvac_sizing.Time, y=hvac_sizing["VAV WITH PFP BOXES AND REHEAT:DesPer 1:Des Heat Cap [W]"], mode='lines+markers', 
            name="Heating Period 1"
            ))
fig.add_trace(
            go.Scatter(x=hvac_sizing.Time, y=hvac_sizing["VAV WITH PFP BOXES AND REHEAT:DesPer 5:Des Heat Cap [W]"], mode='lines+markers', 
            name="Heating Period 5 "
            ))

fig.add_trace(
            go.Scatter(x=hvac_sizing.Time, y=hvac_sizing["VAV WITH PFP BOXES AND REHEAT:DesPer 6:Des Heat Cap [W]"], mode='lines+markers', 
            name="Heating Period 6"
            ))

fig.update_layout(
    title=f"Design Heat Capacity",
    xaxis_title="Hours",
    yaxis_title=f"Capacity [W]",
)
fig.show()

### szs zone based.. sizing

In [8]:
ddySizeFile = os.path.join(new_dir_name, "epluszsz.csv")
des_loads = pd.read_csv(ddySizeFile)
des_loads

,Time,THERMAL ZONE: SPACE 101:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Load [W],THERMAL ZONE: SPACE 101:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Sens Cool Load [W],THERMAL ZONE: SPACE 101:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Mass Flow [kg/s],THERMAL ZONE: SPACE 101:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Cool Mass Flow [kg/s],THERMAL ZONE: SPACE 103:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Load [W],THERMAL ZONE: SPACE 103:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Sens Cool Load [W],THERMAL ZONE: SPACE 103:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Mass Flow [kg/s],THERMAL ZONE: SPACE 103:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Cool Mass Flow [kg/s],THERMAL ZONE: SPACE 201:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Load [W],THERMAL ZONE: SPACE 201:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Sens Cool Load [W],THERMAL ZONE: SPACE 201:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Mass Flow [kg/s],THERMAL ZONE: SPACE 201:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Cool Mass Flow [kg/s],THERMAL ZONE: SPACE 203:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Load [W],THERMAL ZONE: SPACE 203:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Sens Cool Load [W],THERMAL ZONE: SPACE 203:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Mass Flow [kg/s],THERMAL ZONE: SPACE 203:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Cool Mass Flow [kg/s]
0,00:15:00,3699.060,221.34810,0.190891,0.017077,3698.265,220.72280,0.190850,0.017029,8675.887,8175.382,0.447650,0.630706,8675.247,4085.709,0.447617,0.315206
1,00:30:00,3699.076,157.68610,0.190892,0.012166,3698.281,157.09790,0.190851,0.012120,8675.911,8185.302,0.447651,0.631468,8675.271,4096.330,0.447618,0.316023
2,00:45:00,3699.090,88.63092,0.190893,0.006838,3698.295,88.07837,0.190852,0.006795,8675.933,8189.282,0.447652,0.631772,8675.294,4100.986,0.447619,0.316380
3,01:00:00,3699.103,15.47193,0.190893,0.001194,3698.308,14.95502,0.190853,0.001154,8675.954,8188.803,0.447653,0.631733,8675.314,4101.156,0.447620,0.316391
4,01:15:00,3699.126,19.53649,0.190895,0.001507,3698.331,19.05674,0.190854,0.001470,8676.013,8194.041,0.447656,0.632134,8675.373,4106.641,0.447623,0.316813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,23:30:00,3698.972,161.75180,0.190887,0.012481,3698.177,161.14330,0.190846,0.012434,8675.710,8059.089,0.447641,0.621761,8675.070,3967.806,0.447608,0.306120
94,23:45:00,3699.009,235.10260,0.190889,0.018140,3698.214,234.33390,0.190848,0.018080,8675.788,8118.453,0.447645,0.626330,8675.148,4027.660,0.447611,0.310734
95,24:00:00,3699.042,275.62260,0.190890,0.021265,3698.247,274.94570,0.190850,0.021212,8675.860,8157.141,0.447648,0.629305,8675.220,4066.742,0.447615,0.313745
96,Peak,3699.387,11863.71000,0.190908,1.141373,3698.592,11863.81000,0.190867,1.141382,8676.771,12826.340,0.447695,1.226183,8676.131,8690.932,0.447662,0.824221


In [ ]:
des_loads.columns

Index(['Time',
       'THERMAL ZONE: SPACE 101:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Load [W]',
       'THERMAL ZONE: SPACE 101:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Sens Cool Load [W]',
       'THERMAL ZONE: SPACE 101:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Mass Flow [kg/s]',
       'THERMAL ZONE: SPACE 101:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Cool Mass Flow [kg/s]',
       'THERMAL ZONE: SPACE 103:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Load [W]',
       'THERMAL ZONE: SPACE 103:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Sens Cool Load [W]',
       'THERMAL ZONE: SPACE 103:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Mass Flow [kg/s]',
       'THERMAL ZONE: SPACE 103:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Cool Mass Flow [kg/s]',
       'THERMAL ZONE: SPACE 201:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Load [W]',
       'THERMAL ZONE: SPACE 201:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Sens Cool Load

In [15]:
fig = make_subplots(
            rows=1, cols=3,
            subplot_titles=("Heat Mass Flow [kg/s]", 
            "Heat Load [W]", "Cool Load [W]"))


for i in [1,2]:
    for k in [1,3]:
        fig.add_trace(
                    go.Scatter(x=des_loads.Time, y=des_loads[f"THERMAL ZONE: SPACE {i}0{k}:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Mass Flow [kg/s]"], mode='lines+markers', 
                    name=f"Heating Space {i}0{k}",
                    ),row=1, col=1)

for i in [1,2]:
    for k in [1,3]:
        fig.add_trace(
                    go.Scatter(x=des_loads.Time, y=des_loads[f"THERMAL ZONE: SPACE {i}0{k}:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Load [W]"], mode='lines+markers', 
                    name=f"Heating Space {i}0{k}"
                    ), row=1, col=2)

for i in [1,2]:
    for k in [1,3]:
        fig.add_trace(
                    go.Scatter(x=des_loads.Time, y=des_loads[f"THERMAL ZONE: SPACE {i}0{k}:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Sens Cool Load [W]"], mode='lines+markers', 
                    name=f"Cooling Space {i}0{k}"
                    ), row=1, col=3)



fig.show()

In [ ]:
fig = go.Figure()



for i in [1,2]:
    for k in [1,3]:
        fig.add_trace(
                    go.Scatter(x=des_loads.Time, y=des_loads[f"THERMAL ZONE: SPACE {i}0{k}:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Mass Flow [kg/s]"], mode='lines+markers', 
                    name=f"Heating Space {i}0{k}"
                    ))


fig.update_layout(
    title=f"Heating Design Mass Flow",
    xaxis_title="Hours",
    yaxis_title=f"Mass Flow [kg/s]",
)
fig.show()

In [ ]:
fig = go.Figure()



for i in [1,2]:
    for k in [1,3]:
        fig.add_trace(
                    go.Scatter(x=des_loads.Time, y=des_loads[f"THERMAL ZONE: SPACE {i}0{k}:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Sens Cool Load [W]"], mode='lines+markers', 
                    name=f"Cooling Space {i}0{k}"
                    ))


fig.update_layout(
    title=f"Cooling Design Day Load",
    xaxis_title="Hours",
    yaxis_title=f"Load [W]",
)
fig.show()

In [ ]:
fig = go.Figure()



for i in [1,2]:
    for k in [1,3]:
        fig.add_trace(
                    go.Scatter(x=des_loads.Time, y=des_loads[f"THERMAL ZONE: SPACE {i}0{k}:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Load [W]"], mode='lines+markers', 
                    name=f"Heating Space {i}0{k}"
                    ))
        # fig.add_trace(
        #             go.Scatter(x=des_loads.Time, y=des_loads[f"THERMAL ZONE: SPACE {i}0{k}:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Sens Cool Load [W]"], mode='lines+markers', 
        #             name=f"Cooling Space {i}0{k}"
        #             ))


fig.update_layout(
    title=f"Heating Design Day Load",
    xaxis_title="Hours",
    yaxis_title=f"Load [W]",
)
fig.show()